In [ ]:
import signalflow as sf
from pathlib import Path
from datetime import datetime
import polars as pl

## CORE

### Raw Data Load

In [ ]:
spot_store = sf.data.raw_store.DuckDbSpotStore(db_path=Path("test.duckdb"))

loader = sf.data.source.BinanceSpotLoader(store=spot_store)
await loader.download(
    pairs=["BTCUSDT", "ETHUSDT", "SOLUSDT", "BNBUSDT", "XRPUSDT"],
    start=datetime(2025, 12, 1),
    end=datetime(2025, 12, 31),
)

### RawDataFactory

In [ ]:
raw_data = sf.data.RawDataFactory.from_duckdb_spot_store(
    spot_store_path=Path("test.duckdb"),
    pairs=["BTCUSDT", "ETHUSDT", "SOLUSDT"],
    start=datetime(2025, 10, 1),
    end=datetime(2025, 12, 31),
    data_types=["spot"],
)
raw_data_view = sf.core.RawDataView(raw_data)

## FEATURES

In [ ]:
from dataclasses import dataclass
import polars as pl

from signalflow import sf_component
from signalflow.feature.base import Feature


@dataclass
@sf_component(name="custom/log_return")
class CustomLogReturnFeature(Feature):
    """Calculates logarithmic returns: ln(Pt / Pt-1)."""

    price_col: str = "close"
    period: int = 1

    requires = ["{price_col}"]
    outputs = ["log_ret_{period}"]

    def compute_pair(self, df: pl.DataFrame) -> pl.DataFrame:
        return df.with_columns(pl.col(self.price_col).log().diff(n=self.period).alias(f"log_ret_{self.period}"))

In [ ]:
import signalflow as sf

offset_window = 15
pipeline = sf.feature.FeaturePipeline(
    features=[
        sf.feature.ExampleRsiFeature(period=60),
        sf.feature.ExampleRsiFeature(period=180),
        sf.feature.OffsetFeature(
            feature_name="example/rsi",
            feature_params={"period": 180},
            window=offset_window,
            prefix="ofs_",
        ),
        sf.feature.ExampleGlobalMeanRsiFeature(period=60),
        sf.feature.ExampleGlobalMeanRsiFeature(period=180),
        CustomLogReturnFeature(period=60),
        CustomLogReturnFeature(period=180),
    ],
)


features_df = pipeline.run(raw_data_view)
import pandas as pd

pd.set_option("plotting.backend", "plotly")
features_df.filter(pl.col("pair") == "BTCUSDT").select(
    ["rsi_60", "rsi_180", "global_mean_rsi_60", "global_mean_rsi_180", "ofs_rsi_180"]
).to_pandas().plot()

## DETECTOR

### Custom Signal Detector

In [ ]:
from dataclasses import dataclass
import polars as pl

from signalflow import sf_component
from signalflow.core import Signals, SignalType
from signalflow.detector import SignalDetector
from signalflow.feature import FeaturePipeline


@dataclass
@sf_component(name="momentum_breakout")
class CustomMomentumDetector(SignalDetector):
    """Detects large price moves based on log return thresholds."""

    threshold: float = 0.02
    price_col: str = "close"
    period: int = 1

    def __post_init__(self):
        self.feature_col = f"log_ret_{self.period}"

        self.feature_pipeline = FeaturePipeline(
            features=[CustomLogReturnFeature(price_col=self.price_col, period=self.period)]
        )

    def detect(self, features: pl.DataFrame, context: dict | None = None) -> Signals:
        feat = pl.col(self.feature_col)

        out = features.select(
            [
                self.pair_col,
                self.ts_col,
                pl.when(feat > self.threshold)
                .then(pl.lit(SignalType.RISE.value))
                .when(feat < -self.threshold)
                .then(pl.lit(SignalType.FALL.value))
                .otherwise(pl.lit(SignalType.NONE.value))
                .alias("signal_type"),
                pl.when(feat > self.threshold)
                .then(1)
                .when(feat < -self.threshold)
                .then(-1)
                .otherwise(0)
                .alias("signal"),
            ]
        ).filter(pl.col("signal_type") != SignalType.NONE.value)

        return Signals(out)

In [ ]:
detector = CustomMomentumDetector(threshold=0.01)

print("--- Running Pipeline ---")
signals = detector.run(raw_data_view)

print(f"\nDetected {signals.value.height} signals:")
display(signals.value)

print("\nVerifying with Source Data:")
dates_of_interest = signals.value.get_column("timestamp")
spot_df = raw_data.get("spot")

verification = spot_df.filter(pl.col("timestamp").is_in(dates_of_interest))
display(verification)

### Result ploting

In [ ]:
import plotly.graph_objects as go
import polars as pl
import plotly.io as pio


def plot_signals(raw_df: pl.DataFrame, signals_df: pl.DataFrame, pair: str = "BTCUSDT"):

    price_data = raw_df.select(["timestamp", "pair", "close"]).filter(pl.col("pair") == pair)
    sig_data = signals_df.select(["timestamp", "pair", "signal"]).filter(pl.col("pair") == pair)

    price_data = price_data.with_columns(pl.col("timestamp").cast(pl.Datetime("us")))
    sig_data = sig_data.with_columns(pl.col("timestamp").cast(pl.Datetime("us")))

    df_plot = price_data.sort("timestamp").to_pandas()
    signals_with_price = sig_data.join(price_data, on=["timestamp", "pair"], how="inner")
    sig_plot = signals_with_price.to_pandas()
    fig = go.Figure()

    fig.add_trace(
        go.Scatter(
            x=df_plot["timestamp"],
            y=df_plot["close"],
            mode="lines",
            name=f"{pair} Price",
            line=dict(color="#2E86C1", width=1.5),
        )
    )

    buys = sig_plot[sig_plot["signal"] == 1]
    if not buys.empty:
        fig.add_trace(
            go.Scatter(
                x=buys["timestamp"],
                y=buys["close"],
                mode="markers",
                name="Buy Signal",
                marker=dict(symbol="triangle-up", size=13, color="#00CC96", line=dict(width=1, color="black")),
            )
        )

    sells = sig_plot[sig_plot["signal"] == -1]
    if not sells.empty:
        fig.add_trace(
            go.Scatter(
                x=sells["timestamp"],
                y=sells["close"],
                mode="markers",
                name="Sell Signal",
                marker=dict(symbol="triangle-down", size=13, color="#EF553B", line=dict(width=1, color="black")),
            )
        )

    fig.update_layout(
        title=dict(text=f"SignalFlow: {pair} Analysis", font=dict(color="black")),
        xaxis_title="Date",
        yaxis_title="Price",
        template="plotly_white",
        hovermode="x unified",
        height=600,
        xaxis=dict(showgrid=True, gridcolor="lightgray"),
        yaxis=dict(showgrid=True, gridcolor="lightgray"),
    )

    return fig


spot_df = raw_data.get("spot")
fig = plot_signals(spot_df, signals.value, pair="ETHUSDT")
fig.write_image("spot_signals.png")

## VALIDATOR

In [ ]:
from signalflow.target import FixedHorizonLabeler


labeler = FixedHorizonLabeler(price_col="close", horizon=10, include_meta=True)
labeled_df = labeler.compute(df=raw_data_view.to_polars("spot"), signals=signals)

print("Labeled Data Sample (Signals only):")
labeled_signals = labeled_df.filter(pl.col("label") != "none")
display(labeled_signals.select(["timestamp", "label", "ret"]).head())

In [ ]:
from signalflow.validator import SklearnSignalValidator

validator = SklearnSignalValidator(
    model_type="random_forest", model_params={"n_estimators": 100, "max_depth": 5, "random_state": 42}
)


features_df = detector.preprocess(raw_data_view)

split_idx = int(features_df.height * 0.8)
X_train = features_df.slice(0, split_idx)
X_test = features_df.slice(split_idx, None)

y = labeled_df.select("label")
y_train = y.slice(0, split_idx)
y_test = y.slice(split_idx, None)

print("--- Training Validator ---")
validator.fit(X_train, y_train)
print("Model trained successfully.")

In [ ]:
validated_signals = validator.validate_signals(signals, features_df)

print("\nValidated Signals (with probabilities):")
display(
    validated_signals.value.select(["timestamp", "signal_type", "probability_rise", "probability_fall"])
    .sort("probability_rise", descending=True)
    .head()
)

### Visualizing Probability

In [ ]:
import plotly.graph_objects as go
import polars as pl
import signalflow as sf


def plot_strategy_complete(raw_df: pl.DataFrame, val_signals: sf.core.Signals, pair: str = "BTCUSDT"):
    raw_sorted = raw_df.filter(pl.col("pair") == pair).sort("timestamp")
    raw_sorted = raw_sorted.with_columns(pl.col("timestamp").cast(pl.Datetime("us")))
    df_plot = raw_sorted.to_pandas()

    sig_data = val_signals.value.filter(pl.col("pair") == pair)
    sig_data = sig_data.with_columns(pl.col("timestamp").cast(pl.Datetime("us")))

    merged = sig_data.join(raw_sorted.select(["timestamp", "close"]), on="timestamp", how="inner").to_pandas()

    fig = go.Figure()

    fig.add_trace(
        go.Scatter(
            x=df_plot["timestamp"],
            y=df_plot["close"],
            mode="lines",
            name="Price",
            line=dict(color="#2962FF", width=1.5),
        )
    )

    buys = merged[merged["signal_type"] == "rise"]

    if not buys.empty:
        buys_noise = buys[buys["probability_rise"] < 0.5]
        if not buys_noise.empty:
            fig.add_trace(
                go.Scatter(
                    x=buys_noise["timestamp"],
                    y=buys_noise["close"],
                    mode="markers",
                    name="Buy (Ignored)",
                    marker=dict(symbol="triangle-up", size=8, color="#B0BEC5", line=dict(width=1, color="gray")),
                    hovertemplate="<b>Buy (Ignored)</b><br>Price: %{y:.2f}<br>Conf: %{text}<extra></extra>",
                    text=[f"{p:.2f}" for p in buys_noise["probability_rise"]],
                )
            )

        buys_signal = buys[buys["probability_rise"] >= 0.5]
        if not buys_signal.empty:
            sizes = 12 + (buys_signal["probability_rise"] * 15)
            fig.add_trace(
                go.Scatter(
                    x=buys_signal["timestamp"],
                    y=buys_signal["close"],
                    mode="markers",
                    name="Buy (Active)",
                    marker=dict(
                        symbol="triangle-up",
                        size=sizes,
                        color=buys_signal["probability_rise"],
                        colorscale=[[0, "#B9F6CA"], [1, "#00C853"]],
                        cmin=0.5,
                        cmax=1.0,
                        showscale=True,
                        colorbar=dict(title="Buy Conf", x=1.02, len=0.4, y=0.8),
                        line=dict(width=1, color="black"),
                    ),
                    hovertemplate="<b>Buy (Active)</b><br>Price: %{y:.2f}<br>Conf: %{text}<extra></extra>",
                    text=[f"{p:.2f}" for p in buys_signal["probability_rise"]],
                )
            )

    sells = merged[merged["signal_type"] == "fall"]

    if not sells.empty:
        sells_noise = sells[sells["probability_fall"] < 0.5]
        if not sells_noise.empty:
            fig.add_trace(
                go.Scatter(
                    x=sells_noise["timestamp"],
                    y=sells_noise["close"],
                    mode="markers",
                    name="Sell (Ignored)",
                    marker=dict(symbol="triangle-down", size=8, color="#B0BEC5", line=dict(width=1, color="gray")),
                    hovertemplate="<b>Sell (Ignored)</b><br>Price: %{y:.2f}<br>Conf: %{text}<extra></extra>",
                    text=[f"{p:.2f}" for p in sells_noise["probability_fall"]],
                )
            )

        sells_signal = sells[sells["probability_fall"] >= 0.5]
        if not sells_signal.empty:
            sizes = 12 + (sells_signal["probability_fall"] * 15)
            fig.add_trace(
                go.Scatter(
                    x=sells_signal["timestamp"],
                    y=sells_signal["close"],
                    mode="markers",
                    name="Sell (Active)",
                    marker=dict(
                        symbol="triangle-down",
                        size=sizes,
                        color=sells_signal["probability_fall"],
                        colorscale=[[0, "#EF9A9A"], [1, "#C62828"]],
                        cmin=0.5,
                        cmax=1.0,
                        showscale=True,
                        colorbar=dict(title="Sell Conf", x=1.02, len=0.4, y=0.2),
                        line=dict(width=1, color="black"),
                    ),
                    hovertemplate="<b>Sell (Active)</b><br>Price: %{y:.2f}<br>Conf: %{text}<extra></extra>",
                    text=[f"{p:.2f}" for p in sells_signal["probability_fall"]],
                )
            )

    fig.update_layout(
        title=dict(text=f"AI Strategy Analysis: {pair}", font=dict(size=20, color="#333")),
        template="plotly_white",
        height=700,
        hovermode="x unified",
        xaxis=dict(showgrid=True, gridcolor="#eceff1"),
        yaxis=dict(showgrid=True, gridcolor="#eceff1"),
        legend=dict(orientation="h", yanchor="bottom", y=1.02, xanchor="left", x=0),
    )
    fig.show()


plot_strategy_complete(raw_data_view.to_polars("spot"), validated_signals, pair="SOLUSDT")

## STRATEGY

In [ ]:
from signalflow.strategy.broker import BacktestBroker
from signalflow.strategy.broker.executor import VirtualSpotExecutor
from signalflow.data.strategy_store import DuckDbStrategyStore
from signalflow.strategy.runner import BacktestRunner, OptimizedBacktestRunner
from signalflow.strategy.component.entry import SignalEntryRule
from signalflow.strategy.component.exit import TakeProfitStopLossExit
from signalflow.analytic.strategy import (
    TotalReturnMetric,
    BalanceAllocationMetric,
    DrawdownMetric,
    WinRateMetric,
    SharpeRatioMetric,
)

strategy_store = DuckDbStrategyStore("strategy.duckdb")
strategy_store.init()

executor = VirtualSpotExecutor(fee_rate=0.001, slippage_pct=0.001)
broker = BacktestBroker(executor=executor, store=strategy_store)

entry_rule = SignalEntryRule(
    base_position_size=1000.0,
    use_probability_sizing=True,
    min_probability=0.6,
    max_positions_per_pair=1,
    allow_shorts=False,
)

exit_rule = TakeProfitStopLossExit(take_profit_pct=0.02, stop_loss_pct=0.02)

metrics = [
    TotalReturnMetric(initial_capital=10000.0),
    BalanceAllocationMetric(initial_capital=10000.0),
    DrawdownMetric(),
    WinRateMetric(),
    SharpeRatioMetric(initial_capital=10000.0, window_size=100),
]

runner = OptimizedBacktestRunner(
    strategy_id="ml_momentum_strategy",
    broker=broker,
    entry_rules=[entry_rule],
    exit_rules=[exit_rule],
    metrics=metrics,
    initial_capital=10000.0,
    data_key="spot",
)

print("--- Running Backtest ---")
final_state = runner.run(raw_data, validated_signals)

results = runner.get_results()
print(f"\nFinal Equity: ${results['final_equity']:.2f}")
print(f"Total Return: {results['final_return'] * 100:.2f}%")
print(f"Trades Executed: {results['total_trades']}")


print("Recent Trades:")
display(results["trades_df"].tail(10))

metrics_df = results["metrics_df"]
metrics_df

In [ ]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import polars as pl


def plot_backtest_performance(results: dict):

    metrics_df = results.get("metrics_df")

    if metrics_df is None or metrics_df.height == 0:
        print("No metrics to plot")
        return

    if "timestamp" in metrics_df.columns:
        timestamps = (
            metrics_df.select(pl.from_epoch(pl.col("timestamp").cast(pl.Int64), time_unit="s").alias("datetime"))
            .get_column("datetime")
            .to_list()
        )
    else:
        timestamps = list(range(metrics_df.height))

    fig = make_subplots(
        rows=3,
        cols=1,
        shared_xaxes=True,
        vertical_spacing=0.06,
        subplot_titles=("Strategy Performance", "Position Metrics", "Balance Allocation"),
        row_heights=[0.45, 0.30, 0.25],
    )

    if "total_return" in metrics_df.columns:
        returns_pct = (metrics_df.get_column("total_return") * 100).to_list()

        fig.add_trace(
            go.Scatter(
                x=timestamps,
                y=returns_pct,
                mode="lines",
                name="Strategy Return",
                line=dict(color="#1E88E5", width=2.5),
                hovertemplate="Return: %{y:.2f}%<extra></extra>",
            ),
            row=1,
            col=1,
        )

    fig.add_hline(y=0, line_dash="dash", line_color="#9E9E9E", line_width=1, row=1, col=1)

    if "open_positions" in metrics_df.columns:
        open_pos = metrics_df.get_column("open_positions").to_list()

        fig.add_trace(
            go.Scatter(
                x=timestamps,
                y=open_pos,
                mode="lines",
                name="Open Positions",
                line=dict(color="#43A047", width=2),
                fill="tozeroy",
                fillcolor="rgba(67, 160, 71, 0.15)",
                hovertemplate="Open: %{y}<extra></extra>",
            ),
            row=2,
            col=1,
        )

    if "closed_positions" in metrics_df.columns:
        closed_pos = metrics_df.get_column("closed_positions").to_list()

        fig.add_trace(
            go.Scatter(
                x=timestamps,
                y=closed_pos,
                mode="lines",
                name="Closed Positions",
                line=dict(color="#8E24AA", width=1.5, dash="dot"),
                hovertemplate="Closed: %{y}<extra></extra>",
            ),
            row=2,
            col=1,
        )

    if "cash" in metrics_df.columns and "equity" in metrics_df.columns:
        cash = metrics_df.get_column("cash").to_list()
        equity = metrics_df.get_column("equity").to_list()

        initial_capital = results.get("initial_capital", equity[0] if equity else 10000)

        allocated_pct = [(eq - c) / eq if eq > 0 else 0 for eq, c in zip(equity, cash)]
        free_pct = [c / eq if eq > 0 else 0 for eq, c in zip(equity, cash)]

        total_balance_pct = [(eq / initial_capital - 1) * 100 for eq in equity]

        fig.add_trace(
            go.Scatter(
                x=timestamps,
                y=free_pct,
                mode="lines",
                name="Free Cash",
                line=dict(width=0),
                fillcolor="rgba(100, 181, 246, 0.6)",
                fill="tozeroy",
                stackgroup="balance",
                hovertemplate="Free: %{y:.1%}<extra></extra>",
            ),
            row=3,
            col=1,
        )

        fig.add_trace(
            go.Scatter(
                x=timestamps,
                y=allocated_pct,
                mode="lines",
                name="In Positions",
                line=dict(width=0),
                fillcolor="rgba(25, 118, 210, 0.7)",
                fill="tonexty",
                stackgroup="balance",
                hovertemplate="Allocated: %{y:.1%}<extra></extra>",
            ),
            row=3,
            col=1,
        )

        fig.add_trace(
            go.Scatter(
                x=timestamps,
                y=total_balance_pct,
                mode="lines",
                name="Total Return",
                line=dict(color="#1565C0", width=2.5),
                yaxis="y4",
                hovertemplate="Total: %{y:.2f}%<extra></extra>",
            ),
            row=3,
            col=1,
        )

    final_return = results.get("final_return", 0) * 100

    fig.update_layout(
        title=dict(text=f"Backtest Results | Total Return: {final_return:.2f}%", font=dict(size=18, color="#212121")),
        template="plotly_white",
        height=900,
        hovermode="x unified",
        legend=dict(orientation="h", yanchor="bottom", y=1.01, xanchor="left", x=0, font=dict(size=11)),
        showlegend=True,
        plot_bgcolor="#FAFAFA",
        paper_bgcolor="white",
    )

    fig.update_xaxes(title_text="Date", row=3, col=1)
    fig.update_yaxes(title_text="Return (%)", row=1, col=1)
    fig.update_yaxes(title_text="Count", row=2, col=1)
    fig.update_yaxes(title_text="Allocation", row=3, col=1, tickformat=".0%", range=[0, 1])

    fig.update_layout(yaxis4=dict(title="Total Return (%)", overlaying="y3", side="right", showgrid=False))

    fig.update_xaxes(showgrid=True, gridwidth=1, gridcolor="#E0E0E0", showline=True, linewidth=1, linecolor="#BDBDBD")
    fig.update_yaxes(showgrid=True, gridwidth=1, gridcolor="#E0E0E0", showline=True, linewidth=1, linecolor="#BDBDBD")

    return fig


def plot_detailed_metrics(results: dict):

    metrics_df = results.get("metrics_df")

    if metrics_df is None or metrics_df.height == 0:
        print("No metrics to plot")
        return

    if "timestamp" in metrics_df.columns:
        timestamps = (
            metrics_df.select(pl.from_epoch(pl.col("timestamp").cast(pl.Int64), time_unit="s").alias("datetime"))
            .get_column("datetime")
            .to_list()
        )
    else:
        timestamps = list(range(metrics_df.height))

    fig = make_subplots(
        rows=2,
        cols=1,
        shared_xaxes=True,
        vertical_spacing=0.08,
        subplot_titles=("Drawdown Analysis", "Capital Utilization"),
        row_heights=[0.6, 0.4],
    )

    if "current_drawdown" in metrics_df.columns:
        drawdown = metrics_df.get_column("current_drawdown").to_list()
        drawdown_pct = [-d * 100 for d in drawdown]

        fig.add_trace(
            go.Scatter(
                x=timestamps,
                y=drawdown_pct,
                mode="lines",
                name="Drawdown",
                line=dict(color="#E53935", width=2),
                fill="tozeroy",
                fillcolor="rgba(229, 57, 53, 0.2)",
                hovertemplate="DD: %{y:.2f}%<extra></extra>",
            ),
            row=1,
            col=1,
        )

        max_dd = results.get("max_drawdown", 0) * 100
        if max_dd > 0:
            fig.add_hline(
                y=-max_dd,
                line_dash="dash",
                line_color="#C62828",
                line_width=1.5,
                annotation_text=f"Max DD: {max_dd:.2f}%",
                annotation_position="right",
                row=1,
                col=1,
            )

    if "capital_utilization" in metrics_df.columns:
        util = metrics_df.get_column("capital_utilization").to_list()
        util_pct = [u * 100 for u in util]

        fig.add_trace(
            go.Scatter(
                x=timestamps,
                y=util_pct,
                mode="lines",
                name="Capital Utilization",
                line=dict(color="#FB8C00", width=2),
                fill="tozeroy",
                fillcolor="rgba(251, 140, 0, 0.15)",
                hovertemplate="Util: %{y:.1f}%<extra></extra>",
            ),
            row=2,
            col=1,
        )

        fig.add_hline(y=100, line_dash="dot", line_color="#9E9E9E", line_width=1, row=2, col=1)

    fig.update_layout(
        template="plotly_white",
        height=600,
        hovermode="x unified",
        showlegend=True,
        plot_bgcolor="#FAFAFA",
        paper_bgcolor="white",
    )

    fig.update_yaxes(title_text="Drawdown (%)", row=1, col=1)
    fig.update_yaxes(title_text="Utilization (%)", row=2, col=1)
    fig.update_xaxes(title_text="Date", row=2, col=1)

    fig.update_xaxes(showgrid=True, gridwidth=1, gridcolor="#E0E0E0")
    fig.update_yaxes(showgrid=True, gridwidth=1, gridcolor="#E0E0E0")

    return fig


def print_performance_summary(results: dict):

    print("\n" + "=" * 70)
    print("📊 BACKTEST PERFORMANCE SUMMARY")
    print("=" * 70)

    # Performance metrics
    final_return = results.get("final_return", 0) * 100
    max_dd = results.get("max_drawdown", 0) * 100
    sharpe = results.get("sharpe_ratio", 0)

    print(f"\n💰 Returns:")
    print(f"   Total Return:     {final_return:>8.2f}%")
    print(f"   Max Drawdown:     {max_dd:>8.2f}%")
    print(f"   Sharpe Ratio:     {sharpe:>8.3f}")

    if abs(max_dd) > 0.01:
        calmar = final_return / abs(max_dd)
        print(f"   Calmar Ratio:     {calmar:>8.3f}")

    total_trades = results.get("total_trades", 0)
    entry_count = results.get("entry_count", 0)
    exit_count = results.get("exit_count", 0)
    win_rate = results.get("win_rate", 0) * 100 if "win_rate" in results else 0

    print(f"\n📈 Trading Activity:")
    print(f"   Total Trades:     {total_trades:>8}")
    print(f"   Entry Trades:     {entry_count:>8}")
    print(f"   Exit Trades:      {exit_count:>8}")
    if win_rate > 0:
        print(f"   Win Rate:         {win_rate:>8.1f}%")

    final_equity = results.get("final_equity", 0)
    initial_capital = results.get("initial_capital", 10000)

    print(f"\n💵 Capital:")
    print(f"   Initial:          ${initial_capital:>12,.2f}")
    print(f"   Final:            ${final_equity:>12,.2f}")
    print(f"   Profit/Loss:      ${final_equity - initial_capital:>12,.2f}")

    print("\n" + "=" * 70 + "\n")


results = runner.get_results()

print_performance_summary(results)

fig1 = plot_backtest_performance(results)
pio.write_image(fig1, "backtest_performance.png")

fig1.show()

fig2 = plot_detailed_metrics(results)
pio.write_image(fig2, "detailed_metrics.png")

fig2.show()